In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from datetime import datetime
from pandas import to_numeric, to_datetime, to_timedelta, DataFrame
pd.set_option('display.float_format', lambda x: '%.2f' % x)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
def help_func1(m_df):
    cast_list = []
    cast_w_list = []
    i = 0
    while i < (len(m_df)):
        a = list(m_df.iloc[i])
        if "O-O" in a:
            cast_list.append(a.index("O-O")+1)
            cast_w_list.append(1)
            i += 1
        elif "O-O-O" in a:
            cast_list.append(a.index("O-O-O")+1)
            cast_w_list.append(0)
            i += 1
        else:
            cast_list.append(0)
            cast_w_list.append(-1)
            i += 1
    return cast_list, cast_w_list

In [3]:
def help_func2(df, d_df, col_1):
    tmdf = df.shift(1, axis=1).copy()
    tmdf = df-tmdf
    tmdf[col_1] = df[col_1]
    tmdf['tot_time'] = d_df['time_control']
    tmdf['min_time'] = tmdf.min(axis=1)
    return tmdf

In [4]:
def initial_chess_data_cleanup(filename):
    with open(filename, 'r+') as file:
        icd_l = file.readlines()
    chess_text = " ".join(num for num in icd_l if len(num) > 4)
    chess_text = chess_text.replace('[', "").replace(']', "")
    chess_text = chess_text.replace('\n', ' ')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('... ', 'b":"').replace('. ', 'w":"')
    chess_text = chess_text.replace('", ', '", "').replace(' {%clk ', '^')
    chess_text = chess_text.replace(' {%clk', '^')
    chess_text = chess_text.replace('}', '",').replace('", ', '", "')
    chess_text = chess_text.replace(' Site "Chess.com" D', ', D')
    chess_text = chess_text.replace('Event ', '}~{"Event":')
    chess_text = chess_text.replace('", Date ', '", "Date": ')
    chess_text = chess_text.replace('" Result ', '", "Result": ')
    chess_text = chess_text.replace('" Round ', '", "Round": ')
    chess_text = chess_text.replace('" White ', '", "White": ')
    chess_text = chess_text.replace('" Black ', '", "Black": ')
    chess_text = chess_text.replace('" WhiteElo ', '", "WhiteElo": ')
    chess_text = chess_text.replace('" TimeControl ', '", "TimeControl": ')
    chess_text = chess_text.replace('" EndTime ', '", "EndTime": ')
    chess_text = chess_text.replace('" BlackElo ', '", "BlackElo": ').replace(
        '" Termination ', '", "Termination": ')
    chess_text = chess_text.replace(
        '"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "5", "White": "TrueMoeG", "Black": "naggvk", "Result": "0-1", "WhiteElo": "784", "BlackElo": "1210", "TimeControl": "600", "EndTime": "12:27:07 PST", "Termination": "naggvk won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2017.02.20", "Round": "-", "White": "30mate", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "820", "BlackElo": "878", "TimeControl": "600", "EndTime": "11:55:45 PST", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2018.07.14", "Round": "-", "White": "Bran17", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "1205", "BlackElo": "961", "TimeControl": "300", "EndTime": "18:02:56 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2017.04.26", "Round": "-", "White": "nerbenator", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "829", "BlackElo": "842", "TimeControl": "600", "EndTime": "16:49:05 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "4", "White": "TrueMoeG", "Black": "Shuzakhan", "Result": "0-1", "WhiteElo": "845", "BlackElo": "1183", "TimeControl": "600", "EndTime": "11:58:21 PST", "Termination": "Shuzakhan won by resignation" }~{', '')
    chess_text = chess_text.replace(' PST', '').replace(' PDT', '')
    chess_text = chess_text.replace('180+2', '180').replace('300+5', '300')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('" 1w":[', '"}~{"1w":[')
    chess_text = chess_text.replace('" 1w":"', '"}~{"1w":"')
    chess_text = chess_text.replace(', "1/2-1/2 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 }~{', '}~{')
    chess_text = chess_text.replace(', "0-1 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 ', '}').replace(', "}', '}')
    return chess_text

clean_chess_text = initial_chess_data_cleanup('dest.pgn')

In [5]:
def data_cleaning(chess_text):
    cl = ''.join([num for num in chess_text]).split("~")
    df = DataFrame(cl, columns=['a'])
    df = df[df['a'].str.len() > 3]
    c_df = DataFrame(data=list(df['a'].apply(literal_eval)))
    c_df['Date'] = c_df['Date'].fillna(method='ffill')
    c_df['EndTime'] = c_df['EndTime'].fillna(method='ffill')
    c_df['date_time'] = to_datetime(c_df['Date'] + ' ' + c_df['EndTime'])
    c_df['Date'] = to_datetime(c_df['Date'])
    c_df['EndTime'] = to_timedelta(c_df['EndTime'])
    m_df = c_df[c_df.index % 2 == 1]
    m_df = m_df.sort_values('date_time').reset_index().drop(
        columns=['index', 'Date', 'White', 'Black', 'Result', 'WhiteElo',
                 'BlackElo', 'TimeControl', 'EndTime', 'Termination', 'date_time', 'Round'])
    
    moves_column_names = ['00' + num if len(num) == 2 else
                          num for num in m_df.columns]
    moves_column_names = ['0' + num if len(num) == 3 else
                          num for num in moves_column_names]
    moves_column_names = [num.replace('w', 'a') for num
                          in moves_column_names]
    m_df.columns = [num for num in moves_column_names]
    moves_column_names = sorted(moves_column_names)
    m_df = m_df[[num for num in moves_column_names]]
    t_df = m_df.copy()    
    for col_name in m_df.columns:
        m_df[col_name] = m_df[col_name].str.extract(r'(^\w+-?\w+?-?\w?\+?)')
    for col_name in m_df.columns:
        t_df[col_name] = t_df[col_name].str.extract(r'(\d\:\d+:\d+\.?\d)')

    
    d_df = c_df[c_df.index % 2 == 0]
    d_df = d_df[['Date', 'White', 'Black', 'Result', 'WhiteElo', 'BlackElo',
                 'TimeControl', 'EndTime', 'Termination', 'date_time'
                 ]].sort_values('date_time').reset_index().drop(columns=[
                     'index', 'date_time'])
    d_df = d_df.rename(
        columns={'Date': 'date', 'White': 'white', 'Black': 'black', 'Result': 'result',
                 'WhiteElo': 'white_elo', 'BlackElo': 'black_elo', 'TimeControl': 'time_control',
                 'EndTime': 'end_time', 'Termination': 'termination'})
    d_df['num_moves'] = m_df.count(axis=1)
    d_df['time_control'] = to_numeric(d_df['time_control'])
    d_df['white_elo'] = to_numeric(d_df['white_elo'])
    d_df['black_elo'] = to_numeric(d_df['black_elo'])
    d_df['color'] = np.where(d_df['white'] == 'TrueMoeG', 1, 0)
    

    t_df = t_df.apply(pd.to_timedelta, errors='coerce')
    t_df = t_df.apply(pd.to_numeric, errors='coerce')
    t_df = t_df.div(1000000000)
    for col_name in t_df.columns:
        t_df[col_name] = d_df['time_control'] - t_df[col_name]
    t_df[t_df>605] = 0
    wh_m_df = m_df[m_df.columns[::2]].copy()
    bl_m_df = m_df[m_df.columns[1::2]].copy()
    wh_t_df = t_df[t_df.columns[::2]].copy()
    bl_t_df = t_df[t_df.columns[1::2]].copy()

    wh_t_df['tot_time'] = d_df['time_control']
    bl_t_df['tot_time'] = d_df['time_control']
    d_df['white_num_moves'] = wh_m_df.count(axis=1)
    d_df['black_num_moves'] = bl_m_df.count(axis=1)
    
    wh_tm_df = help_func2(wh_t_df, d_df, '001a')
    bl_tm_df = help_func2(bl_t_df, d_df, '001b')
    
    d_df['white_time_used'] = wh_tm_df.max(axis=1)
    d_df['black_time_used'] = bl_tm_df.max(axis=1)
    
    d_df['winner'] = d_df['termination'].str.extract('(^[a-zA-Z0-9]+)', expand=False)
    d_df['won_by'] = d_df['termination'].str.extract('([a-zA-Z0-9]+$)', expand=False)
    
    cstl_l_bl, cstl_loc_l_bl = help_func1(bl_m_df)
    cstl_l_wh, cstl_loc_l_wh = help_func1(wh_m_df)
    
    d_df['weekday'] = d_df.date.apply(lambda x: x.dayofweek)
    d_df['day'] = d_df.date.apply(lambda x: x.day)
    
    d_df['result'] = np.where(d_df['winner'] == 'TrueMoeG',
                              1.0, (np.where(d_df['winner'] == 'Game', 0.5, 0.0)))
    
    d_df['white_castled_on'] = pd.Series(cstl_l_wh)
    d_df['black_castled_on'] = pd.Series(cstl_l_bl)
    d_df['white_castled_where'] = pd.Series(cstl_loc_l_wh)
    d_df['black_castled_where'] = pd.Series(cstl_loc_l_bl)
    
    d_df['castled_on'] = np.where(d_df['color'] == 1, d_df[
        'white_castled_on'], d_df['black_castled_on'])
    d_df['opp_castled_on'] = np.where(d_df['color'] == 0, d_df[
        'white_castled_on'], d_df['black_castled_on'])
    d_df['castled'] = np.where(d_df['color'] == 1, d_df[
        'white_castled_where'], d_df['black_castled_where'])
    d_df['opp_castled'] = np.where(d_df['color'] == 0, d_df[
        'white_castled_where'], d_df['black_castled_where'])
    d_df = d_df.drop(columns=['white_castled_on', 'black_castled_on', 
                              'white_castled_where', 'black_castled_where'])
    
    d_df['time_used'] = np.where(d_df['color'] == 1, d_df[
        'white_time_used'], d_df['black_time_used'])    
    d_df['opp_time_used'] = np.where(d_df['color'] == 0, d_df[
        'white_time_used'], d_df['black_time_used'])
    
    d_df['time_control'] = np.where(d_df['time_control'] >= d_df[
        'time_used'], d_df['time_control'], d_df['time_used'])    
    d_df['time_control'] = np.where(d_df['time_control'] >= d_df[
        'opp_time_used'], d_df['time_control'], d_df['opp_time_used'])
    
    d_df['time_used'] = np.where(d_df['result'] == 1.0, d_df[
        'time_used'], np.where(d_df['won_by'] == 'time', d_df[
            'time_control'], d_df['time_used']))    
    d_df['opp_time_used'] = np.where(d_df['result'] == 0.0, d_df[
        'opp_time_used'], np.where(d_df['won_by'] == 'time', d_df[
            'time_control'], d_df['opp_time_used']))
    
    d_df['num_moves'] = np.where(d_df['color'] == 1, d_df[
        'white_num_moves'], d_df['black_num_moves'])
    d_df['opp_num_moves'] = np.where(d_df['color'] == 0, d_df[
        'white_num_moves'], d_df['black_num_moves'])
    
    d_df['avg_time'] = d_df['time_used']/d_df['num_moves']
    d_df['opp_avg_time'] = d_df['opp_time_used']/d_df['opp_num_moves']
    d_df['white_max_move'] = pd.Series(list(wh_tm_df.max(axis=1)))
    d_df['black_max_move'] = pd.Series(list(bl_tm_df.max(axis=1)))
    d_df['max_time'] = np.where(d_df['color'] == 1, d_df[
        'white_max_move'], d_df['black_max_move'])
    d_df['opp_max_time'] = np.where(d_df['color'] == 0, d_df[
        'white_max_move'], d_df['black_max_move'])
    d_df['post_elo'] = np.where(d_df['color'] == 1, d_df[
        'white_elo'], d_df['black_elo'])
    d_df['end_time'] = to_numeric(d_df['end_time'])/3600000000
    d_df['start_time'] = (d_df['end_time'])-(d_df['white_time_used']+d_df['black_time_used'])/3.6
    
    d_df['opp_post_elo'] = np.where(d_df['color'] == 0, d_df[
        'white_elo'], d_df['black_elo'])
    d_df['elo'] = np.where(d_df['post_elo'] > 0, d_df['post_elo'], 900)

    d_df['elo_delta'] = d_df['elo'] - d_df['post_elo']
    d_df['opp_elo'] = d_df['opp_post_elo'] + d_df['elo_delta']
    d_df['diff'] = d_df['elo'] - d_df['opp_elo']
    
    
#     d_df = d_df.drop(columns=[
#         'termination', 'winner', 'post_elo','opp_post_elo', 'elo_delta', 'white_elo',
#         'black_elo', 'white', 'black', 'white_time_used','black_time_used', 
#         'white_max_move', 'black_max_move', 'white_num_moves', 'black_num_moves'])
    return d_df[['result', 'elo', 'opp_elo', 'diff', 'color', 'time_control', 'start_time', 'weekday', 'day','white_time_used','black_time_used','end_time']]


In [ ]:
ddf = data_cleaning(clean_chess_text)
ddf

In [ ]:
ddf['end_time'] = to_numeric(ddf['end_time'])/3600000000
ddf.end_time.value_counts()